https://www.kaggle.com/competitions/ieee-fraud-detection

Este notebook engloba os seguinte passos:

* Análise Exploratória de Dados e pontuações sobre

# Sobre os Dados

## Tabela Transação

* `TransactionDT`: timedelta from a given reference datetime (not an actual timestamp)

* `TransactionAMT`: transaction payment amount in USD

* `ProductCD`: product code, the product for each transaction

* `card1 - card6`: payment card information, such as card type, card category, issue bank, country, etc.

* `addr`: address

* `dist`: distance

* `P_ and (R__) emaildomain`: purchaser and recipient email domain

* `C1-C14`: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.

* `D1-D15`: timedelta, such as days between previous transaction, etc.

* `M1-M9`: match, such as names on card and address, etc.

* `Vxxx`: Vesta engineered rich features, including ranking, counting, and other entity relations.

### Categorical Features

* `ProductCD`

* `card1 - card6`

* `addr1, addr2`

* `P_emaildomain`

* `R_emaildomain`

* `M1 - M9`

## Tabela Identidade

Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.

They're collected by Vesta’s fraud protection system and digital security partners.
(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

### Categorical Features

* `DeviceType`

* `DeviceInfo`

* `id_12 - id_38`

# Importando Bibliotecas

In [13]:
!pip install lightgbm
!pip install xgboost
!pip install catboost

!pip install scikit-learn==1.2.2
!pip install numpy==1.24.0
!pip install pandas==2.0.2
!pip install imbalanced-learn==0.10.1

In [14]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, log_loss, roc_auc_score, accuracy_score
from scipy import stats
pd.options.display.max_rows = 999
pd.options.display.precision = 15

#import lightgbm as lgb
#import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, \
KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import VarianceThreshold
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use("ggplot")
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

In [15]:
start_time = time.time()

In [16]:
%%time

#limpando cache do CUDA
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

SystemError: initialization of _internal failed without raising an exception

In [17]:
# checar utilização de memória
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
gpu_usage() 

| ID | GPU | MEM |
------------------


In [18]:
#limpando cache do CUDA
!pip install torch
import torch
torch.cuda.empty_cache()

# Importando Dados

In [19]:
t0 = time.time()

In [20]:
folder_path = ''
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
sub = pd.read_csv(f'{folder_path}sample_submission.csv')

# combinando dados de transações e de identidade
train = pd.merge(train_transaction, train_identity, on = 'TransactionID', how = 'left')
test = pd.merge(test_transaction, test_identity, on = 'TransactionID', how = 'left')

In [21]:
print(f'Train dataset has {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'Test dataset has {test.shape[0]} rows and {test.shape[1]} columns.')

Train dataset has 590540 rows and 434 columns.
Test dataset has 506691 rows and 433 columns.


# Análise Exploratória de Dados

## Estatística Básica

Olhando as estatísticas básicas através da função `resumetable`

In [22]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = \
        round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

In [23]:
resumetable(train)[:25]

Dataset Shape: (590540, 434)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,TransactionID,int64,0,590540,2987000,2987001,2987002,19.170000000000002
1,isFraud,int64,0,2,0,0,0,0.220000000000000
2,TransactionDT,int64,0,573349,86400,86401,86469,19.109999999999999
3,TransactionAmt,float64,0,20902,68.5,29.0,59.0,8.350000000000000
4,ProductCD,object,0,5,W,W,W,1.280000000000000
5,card1,int64,0,13553,13926,2755,4663,9.970000000000001
6,card2,float64,8933,500,NaN,404.0,490.0,6.320000000000000
7,card3,float64,1565,114,150.0,150.0,150.0,0.680000000000000
8,card4,object,1577,4,discover,mastercard,visa,1.090000000000000
9,card5,float64,4259,119,142.0,102.0,166.0,2.660000000000000


In [ ]:
resumetable(test)[:25]

In [ ]:
# checando o desbalanceamento dos dados através da contagem de linhas e TransactionAmt
train['TransactionAmt'] = train['TransactionAmt'].astype(float)
total = len(train)
total_amt = train.groupby(['isFraud'])['TransactionAmt'].sum().sum()
plt.figure(figsize = (16, 6))

plt.subplot(121)
g = sns.countplot(x = 'isFraud', data = train, )
g.set_title("Fraud Transactions Distribution \n# 0: No Fraud | 1: Fraud #", fontsize = 22)
g.set_xlabel("Is Fraud?", fontsize = 18)
g.set_ylabel('Count', fontsize = 18)

for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width()/2.,
          height + 3,
          '{:1.2f}%'.format(height/total * 100),
          ha = "center", fontsize = 15)

perc_amt = (train.groupby(['isFraud'])['TransactionAmt'].sum())
perc_amt = perc_amt.reset_index()
plt.subplot(122)
g1 = sns.barplot(x = 'isFraud', y = 'TransactionAmt', dodge = True, data = perc_amt)
g1.set_title("% Total Amount in Transaction Amt \n# 0: No Fraud | 1: Fraud #", fontsize = 22)
g1.set_xlabel("Is Fraud?", fontsize = 18)
g1.set_ylabel('Total Transaction Amount Scalar', fontsize = 18)
for p in g1.patches:
    height = p.get_height()
    g1.text(p.get_x() + p.get_width()/2.,
           height + 3,
           '{:1.2f}%'.format(height/total_amt * 100),
           ha = "center", fontsize = 15)
    
plt.show()

In [ ]:
# checando se existem colunas apenas com um valor
one_value_cols_train = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]

print(f'There are {len(one_value_cols_train)} columns in train dataset with one unique value.')
print(f'There are {len(one_value_cols_test)} columns in test dataset with one unique value.')

In [ ]:
# função para checar valores NaN
def NaN_percent(df, column_name):
    #me dá a posição 0 do resultado de df.shape
    row_count = df[column_name].shape[0]
    #df.count() conta os valores não vazios
    empty_values = row_count - df[column_name].count()
    return (100.0 * empty_values)/row_count

In [ ]:
for i in list(train):
    #str() converte número em string
    print(i + ': ' + str(NaN_percent(train,i))+'%')

In [ ]:
for i in list(train):
    #str() converte número em string
    print("'" + i + "'" + ", ")

## Gráficos

### Tabela Identidade

Utilizando a visão de gráficos para apoiar a parte de Feature Engineering

In [ ]:
def cat_feat_ploting(df, col):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

    plt.figure(figsize=(14,10))
    plt.suptitle(f'{col} Distributions', fontsize=22)

    plt.subplot(221)
    g = sns.countplot(x=col, data=df, order=tmp[col].values)
    # plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])

    g.set_title(f"{col} Distribution", fontsize=19)
    g.set_xlabel(f"{col} Name", fontsize=17)
    g.set_ylabel("Count", fontsize=17)
    # g.set_ylim(0,500000)
    for p in g.patches:
        height = p.get_height()
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center", fontsize=14) 

    plt.subplot(222)
    g1 = sns.countplot(x=col, hue='isFraud', data=df, order=tmp[col].values)
    plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
    gt = g1.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, color='black', order=tmp[col].values)#, legend=False)
    gt.set_ylabel("% of Fraud Transactions", fontsize=16)

    g1.set_title(f"{col} by Target(isFraud)", fontsize=19)
    g1.set_xlabel(f"{col} Name", fontsize=17)
    g1.set_ylabel("Count", fontsize=17)

    plt.subplot(212)
    g3 = sns.boxenplot(x=col, y='TransactionAmt', hue='isFraud', 
                       data=df[df['TransactionAmt'] <= 2000], order=tmp[col].values )
    g3.set_title("Transaction Amount Distribuition by ProductCD and Target", fontsize=20)
    g3.set_xlabel("ProductCD Name", fontsize=17)
    g3.set_ylabel("Transaction Values", fontsize=17)

    plt.subplots_adjust(hspace = 0.4, top = 0.85)

    plt.show()

In [ ]:
for col in ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 
            'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']:
    train[col] = train[col].fillna('NaN')
    cat_feat_ploting(train, col)

Features com alguns padrões:

* `id_12`, `id_15`, `id_16`, `id_23`, `id_27`, `id_28`, `id_29`, `id_32`, `id_34`, `id_35`, `id_36`, `id_37`, `id_38`: `NaN` corresponde a mais de 75% dos valores

* `id_13`, `id_14`, `id_17`, `id_18`, `id_19`, `id_20`, `id_21`, `id_22`, `id_24`, `id_25`, `id_26`, `id_30`, `id_31`, `id_33`, `id_37`: pequena concentração dos valores em grupos

#### Operating System

Apesar das pequenas concentrações, é possível concentrar os valores de algumas colunas

In [ ]:
train.loc[train['id_30'].str.contains('Windows', na = False), 'id_30'] = 'Windows'
train.loc[train['id_30'].str.contains('iOS', na = False), 'id_30'] = 'iOS'
train.loc[train['id_30'].str.contains('Mac OS', na = False), 'id_30'] = 'Mac'
train.loc[train['id_30'].str.contains('Android', na = False), 'id_30'] = 'Android'
train['id_30'].fillna("NAN", inplace = True)

In [ ]:
def ploting_cnt_amt(df, col, lim=2000):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)
    
    plt.figure(figsize=(16,14))    
    plt.suptitle(f'{col} Distributions ', fontsize=24)
    
    plt.subplot(211)
    g = sns.countplot( x=col,  data=df, order=list(tmp[col].values))
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values),
                       color='black')#, legend=False)
    gt.set_ylim(0,tmp['Fraud'].max()*1.1)
    gt.set_ylabel("%Fraud Transactions", fontsize=16)
    g.set_title(f"Most Frequent {col} values and % Fraud Transactions", fontsize=20)
    g.set_xlabel(f"{col} Category Names", fontsize=16)
    g.set_ylabel("Count", fontsize=17)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    sizes = []
    for p in g.patches:
        height = p.get_height()
        sizes.append(height)
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center",fontsize=12) 
        
    g.set_ylim(0,max(sizes)*1.15)
    
    #########################################################################
    perc_amt = (df.groupby(['isFraud',col])['TransactionAmt'].sum() \
                / df.groupby([col])['TransactionAmt'].sum() * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)
    amt = df.groupby([col])['TransactionAmt'].sum().reset_index()
    perc_amt = perc_amt.fillna(0)
    plt.subplot(212)
    g1 = sns.barplot(x=col, y='TransactionAmt', 
                       data=amt, 
                       order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt, 
                        order=list(tmp[col].values),
                       color='black')#, legend=False)
    g1t.set_ylim(0,perc_amt['Fraud'].max()*1.1)
    g1t.set_ylabel("%Fraud Total Amount", fontsize=16)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    g1.set_title(f"{col} by Transactions Total + %of total and %Fraud Transactions", fontsize=20)
    g1.set_xlabel(f"{col} Category Names", fontsize=16)
    g1.set_ylabel("Transaction Total Amount(U$)", fontsize=16)
    g1.set_xticklabels(g.get_xticklabels(),rotation=45)    
    
    for p in g1.patches:
        height = p.get_height()
        g1.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total_amt*100),
                ha="center",fontsize=12) 
        
    plt.subplots_adjust(hspace=.4, top = 0.9)
    plt.show()

In [ ]:
ploting_cnt_amt(train, 'id_30')

Mesmo com a concentração, `id_30` demonstra muitos valores `NaN`. Faremos o mesmo para outras colunas da tabela Identidade

#### Browser column

In [ ]:
train.loc[train['id_31'].str.contains('chrome', na=False), 'id_31'] = 'Chrome'
train.loc[train['id_31'].str.contains('firefox', na=False), 'id_31'] = 'Firefox'
train.loc[train['id_31'].str.contains('safari', na=False), 'id_31'] = 'Safari'
train.loc[train['id_31'].str.contains('edge', na=False), 'id_31'] = 'Edge'
train.loc[train['id_31'].str.contains('ie', na=False), 'id_31'] = 'IE'
train.loc[train['id_31'].str.contains('samsung', na=False), 'id_31'] = 'Samsung'
train.loc[train['id_31'].str.contains('opera', na=False), 'id_31'] = 'Opera'
train['id_31'].fillna("NAN", inplace=True)
train.loc[train.id_31.isin(train.id_31.value_counts()[train.id_31.value_counts() 
                                                               < 200].index), 'id_31'] = "Others"

In [ ]:
ploting_cnt_amt(train, 'id_31')

#### Device Type

In [ ]:
train.groupby('DeviceType') \
.mean()['isFraud'] \
.sort_values() \
.plot(kind = 'barh', 
     figsize = (15, 5), 
     title = 'Percentage of Fraud by Device Type')
plt.show()

#### Device Info

In [ ]:
train.groupby('DeviceInfo') \
.count()['TransactionID'] \
.sort_values(ascending = False) \
.head(20) \
.plot(kind = 'barh', figsize = (15, 5), title = 'Top 20 Devices in Train')
plt.show()

### Tabela Transação

#### Transaction Amount

In [ ]:
train['TransactionAmt'] = train['TransactionAmt'].astype(float)
print("Transaction Amount Quantiles: ")
print(train['TransactionAmt'].quantile([.01, .025, .1, .25, .5, .75, .9, .975, .99]))

In [ ]:
# observando os percentis para transações fraudulentas e genuínas
print(pd.concat([train[train['isFraud'] == 1]['TransactionAmt']
                 .quantile([.01, .1, .25, .5, .75, .9, .99])
                 .reset_index(),
                 train[train['isFraud'] == 0]['TransactionAmt']
                 .quantile([.01, .1, .25, .5, .75, .9, .99])
                 .reset_index()], 
                axis = 1, keys = ['Fraud', 'NoFraud']))

In [ ]:
print('Mean transaction amt for fraud is {:.4f}'
      .format(train.loc[train['isFraud'] == 1]['TransactionAmt'].mean()))
print('Mean transaction amt for non-fraud is {:.4f}'
      .format(train.loc[train['isFraud'] == 0]['TransactionAmt'].mean()))

#### Product Feature

In [ ]:
tmp = pd.crosstab(train['ProductCD'], train['isFraud'], normalize = 'index') * 100
tmp = tmp.reset_index()
tmp.rename(columns = {0: 'No Fraud', 1: 'Fraud'}, inplace = True)

plt.figure(figsize = (14, 10))
plt.suptitle('ProductCD Distributions', fontsize = 22)

plt.subplot(221)
g = sns.countplot(x = 'ProductCD', data = train)
# plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])

g.set_title("ProductCD Distribution", fontsize = 19)
g.set_xlabel("ProductCD Name", fontsize = 17)
g.set_ylabel("Count", fontsize = 17)
g.set_ylim(0, 500000)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width()/2.,
          height + 3, 
          '{:1.2f}%'.format(height / total * 100),
          ha = "center", fontsize = 14)

plt.subplot(222)
g1 = sns.countplot(x = 'ProductCD', hue = 'isFraud', data = train)
plt.legend(title = 'Fraud', loc = 'best', labels = ['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x = 'ProductCD', y = 'Fraud', data = tmp, color = 'black', 
                  order = ['W', 'H', 'C', 'S', 'R'])#, legend = False)
gt.set_ylabel("% of Fraud Transactions", fontsize = 16)

g1.set_title("ProductCD by Target (isFraud)", fontsize = 19)
g1.set_xlabel("ProductCD Name", fontsize = 17)
g1.set_ylabel("Count", fontsize = 17)

plt.subplot(212)
g3 = sns.boxenplot(x = 'ProductCD', y = 'TransactionAmt', hue = 'isFraud',
                  data = train[train['TransactionAmt'] <= 2000])
g3.set_title("Transaction Amount Distribution by ProductCD and Target", fontsize = 20)
g3.set_xlabel("ProductCD Name", fontsize = 17)
g3.set_ylabel("Transaction Values", fontsize = 17)

plt.subplots_adjust(hspace = 0.6, top = 0.85)

plt.show()

In [ ]:
train.groupby('ProductCD') \
['TransactionID'].count() \
.sort_index() \
.plot(kind = 'barh', 
     figsize = (15, 3), 
     title = 'Percentage of Fraud by ProductCD')
plt.show()

A maior parte das transações estão relacionadas com o produto `W`

#### Card Features

In [ ]:
resumetable(train[['card1', 'card2', 'card3', 'card4', 'card5', 'card6']])

In [ ]:
# percentis para as colunas de cartão
print("Card Features Quantiles: ")
print(train[['card1', 'card2', 'card3', 'card5']].quantile([.01, .025, .1, .25, .5, .75, .975, .99]))

`Card 1` e `Card 2` com uma ampla distribuição de valores. Melhor tirar o log dessas colunas

In [ ]:
train.loc[train.card3.isin(train.card3.value_counts()[train.card3.value_counts() < 200]
                                 .index), 'card3'] = "Others"
train.loc[train.card5.isin(train.card5.value_counts()[train.card5.value_counts() < 300]
                                .index), 'card5'] = "Others"

In [ ]:
# visualizando as distribuições de Card 1, Card 2 e Card 3
tmp = pd.crosstab(train['card3'], train['isFraud'], normalize='index') * 100
tmp = tmp.reset_index()
tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

tmp2 = pd.crosstab(train['card5'], train['isFraud'], normalize='index') * 100
tmp2 = tmp2.reset_index()
tmp2.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

plt.figure(figsize=(14,22))

plt.subplot(411)
g = sns.distplot(train[train['isFraud'] == 1]['card1'], label='Fraud')
g = sns.distplot(train[train['isFraud'] == 0]['card1'], label='NoFraud')
g.legend()
g.set_title("Card 1 Values Distribution by Target", fontsize=20)
g.set_xlabel("Card 1 Values", fontsize=18)
g.set_ylabel("Probability", fontsize=18)

plt.subplot(412)
g1 = sns.distplot(train[train['isFraud'] == 1]['card2'].dropna(), label='Fraud')
g1 = sns.distplot(train[train['isFraud'] == 0]['card2'].dropna(), label='NoFraud')
g1.legend()
g1.set_title("Card 2 Values Distribution by Target", fontsize=20)
g1.set_xlabel("Card 2 Values", fontsize=18)
g1.set_ylabel("Probability", fontsize=18)

plt.subplot(413)
g2 = sns.countplot(x='card3', data=train, order=list(tmp.card3.values))
g22 = g2.twinx()
gg2 = sns.pointplot(x='card3', y='Fraud', data=tmp, 
                    color='black', order=list(tmp.card3.values))
gg2.set_ylabel("% of Fraud Transactions", fontsize=16)
g2.set_title("Card 3 Values Distribution and % of Transaction Frauds", fontsize=20)
g2.set_xlabel("Card 3 Values", fontsize=18)
g2.set_ylabel("Count", fontsize=18)
for p in g2.patches:
    height = p.get_height()
    g2.text(p.get_x()+p.get_width()/2.,
            height + 25,
            '{:1.2f}%'.format(height/total*100),
            ha="center") 

plt.subplot(414)
g3 = sns.countplot(x='card5', data=train, order=list(tmp2.card5.values))
g3t = g3.twinx()
g3t = sns.pointplot(x='card5', y='Fraud', data=tmp2, 
                    color='black', order=list(tmp2.card5.values))
g3t.set_ylabel("% of Fraud Transactions", fontsize=16)
g3.set_title("Card 5 Values Distribution and % of Transaction Frauds", fontsize=20)
g3.set_xticklabels(g3.get_xticklabels(),rotation=90)
g3.set_xlabel("Card 5 Values", fontsize=18)
g3.set_ylabel("Count", fontsize=18)
for p in g3.patches:
    height = p.get_height()
    g3.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center",fontsize=11) 
    
plt.subplots_adjust(hspace = 0.6, top = 0.85)


plt.show()

In [ ]:
# Card 4
tmp = pd.crosstab(train['card4'], train['isFraud'], normalize = 'index') * 100
tmp = tmp.reset_index()
tmp.rename(columns = {0: 'No Fraud', 1: 'Fraud'}, inplace = True)

plt.figure(figsize = (14, 10))
plt.suptitle('Card 4 Distributions', fontsize = 22)

plt.subplot(221)
g = sns.countplot(x = 'card4', data = train)
# plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])
g.set_title("Card4 Distribution", fontsize = 19)
g.set_ylim(0, 420000)
g.set_xlabel("Card4 Category Names", fontsize = 17)
g.set_ylabel("Count", fontsize = 17)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x() + p.get_width()/2.,
          height + 3,
          '{:1.2f}%'.format(height/total * 100),
          ha = "center", fontsize = 14)

plt.subplot(222)
g1 = sns.countplot(x = 'card4', hue = 'isFraud', data = train)
plt.legend(title = 'Fraud', loc = 'best', labels = ['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x = 'card4', y = 'Fraud', data = tmp,
                  color = 'black',
                  order = ['discover', 'mastercard', 'visa', 'american express'])#, legend = False)
gt.set_ylabel("% of Fraud Transactions", fontsize = 16)
g1.set_title("Card4 by Target(isFraud)", fontsize = 19)
g1.set_xlabel("Card4 Category Names", fontsize = 17)
g1.set_ylabel("Count", fontsize = 17)

plt.subplot(212)
g3 = sns.boxenplot(x = 'card4', y = 'TransactionAmt', hue = 'isFraud', 
                   data = train[train['TransactionAmt'] <= 2000])
g3.set_title("Card4 Distribution by ProductCD and Target", fontsize = 20)
g3.set_xlabel("Card4 Category Names", fontsize = 17)
g3.set_ylabel("Transaction Values", fontsize = 17)

plt.subplots_adjust(hspace = 0.6, top = 0.85)

plt.show()

* 97% dos dados de `Card 4` são Mastercard(32%) e Visa(65%);

* As maiores taxas de fraude estão em Discover (~8%), contra ~3.5% de Mastercard + Visa e 2.87% in American Express

In [ ]:
# Card 6
tmp = pd.crosstab(train['card6'], train['isFraud'], normalize = 'index') * 100
tmp = tmp.reset_index()
tmp.rename(columns = {0: 'NoFraud', 1: 'Fraud'}, inplace = True)

plt.figure(figsize = (14, 10))
plt.suptitle('Card6 Distributions', fontsize = 22)

plt.subplot(221)
g = sns.countplot(x = 'card6', data = train, order = list(tmp.card6.values))
# plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])
g.set_title("Card6 Distribution", fontsize = 19)
g.set_ylim(0, 480000)
g.set_xlabel("Card6 Category Names", fontsize = 17)
g.set_ylabel("Count", fontsize = 17)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center",fontsize=14) 

plt.subplot(222)
g1 = sns.countplot(x='card6', hue='isFraud', data=train, order=list(tmp.card6.values))
plt.legend(title='Fraud', loc='best', labels=['No', 'Yes'])
gt = g1.twinx()
gt = sns.pointplot(x='card6', y='Fraud', data=tmp, order=list(tmp.card6.values),
                   color='black')#, legend=False)
gt.set_ylim(0,20)
gt.set_ylabel("% of Fraud Transactions", fontsize=16)
g1.set_title("Card6 by Target(isFraud)", fontsize=19)
g1.set_xlabel("Card6 Category Names", fontsize=17)
g1.set_ylabel("Count", fontsize=17)

plt.subplot(212)
g3 = sns.boxenplot(x='card6', y='TransactionAmt', hue='isFraud', order=list(tmp.card6.values),
              data=train[train['TransactionAmt'] <= 2000] )
g3.set_title("Card 6 Distribuition by ProductCD and Target", fontsize=20)
g3.set_xlabel("Card6 Category Names", fontsize=17)
g3.set_ylabel("Transaction Values", fontsize=17)

plt.subplots_adjust(hspace = 0.6, top = 0.85)

plt.show()

* 89% dos dados são de cartão de crédito (25%) e débito (74%);

* Maiores taxas de fraudes em débito (~20%) contra 5% de crédito

#### M1-M9

In [ ]:
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    train[col] = train[col].fillna("Miss")
def ploting_dist_ratio(df, col, lim=2000):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

    plt.figure(figsize=(20,5))
    plt.suptitle(f'{col} Distributions ', fontsize=22)

    plt.subplot(121)
    g = sns.countplot(x=col, data=df, order=list(tmp[col].values))
    # plt.legend(title='Fraud', loc='upper center', labels=['No', 'Yes'])
    g.set_title(f"{col} Distribution\nCound and %Fraud by each category", fontsize=18)
    g.set_ylim(0,400000)
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values),
                       color='black')#, legend=False)
    gt.set_ylim(0,20)
    gt.set_ylabel("% of Fraud Transactions", fontsize=16)
    g.set_xlabel(f"{col} Category Names", fontsize=16)
    g.set_ylabel("Count", fontsize=17)
    for p in gt.patches:
        height = p.get_height()
        gt.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center",fontsize=14) 
        
    perc_amt = (train.groupby(['isFraud',col])['TransactionAmt'].sum() / 
                total_amt * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)

    plt.subplot(122)
    g1 = sns.boxplot(x=col, y='TransactionAmt', hue='isFraud', 
                     data=df[df['TransactionAmt'] <= lim], order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt, order=list(tmp[col].values),
                       color='black')#, legend=False)
    g1t.set_ylim(0,5)
    g1t.set_ylabel("%Fraud Total Amount", fontsize=16)
    g1.set_title(f"{col} by Transactions dist", fontsize=18)
    g1.set_xlabel(f"{col} Category Names", fontsize=16)
    g1.set_ylabel("Transaction Amount(U$)", fontsize=16)
        
    plt.subplots_adjust(hspace=.4, wspace = 0.35, top = 0.80)
    
    plt.show()

In [ ]:
# distribuições M: distribuição de contagem, percentual de fraude e valor total de transação
for col in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']:
    ploting_dist_ratio(train, col, lim=2500)

Em todos os casos, mais da metade das transações tem valores faltantes.

#### Addr1 and Addr2

In [ ]:
print("Card Features Quantiles")
print(train[['addr1', 'addr2']].quantile([0.01, .025, .1, .25, .5, .75, .90, .975, .99]))

In [ ]:
train.loc[train.addr1.isin(train.addr1.value_counts()[train.addr1.value_counts() <= 
                                                               5000].index), 'addr1'] = "Others"
train.loc[train.addr2.isin(train.addr2.value_counts()[train.addr2.value_counts() <= 
                                                              50].index), 'addr2'] = "Others"

In [ ]:
# distribuição de addr1
ploting_cnt_amt(train, 'addr1')

Baixa concentração verificada na feature `addr1`

In [ ]:
# Addr2 Distributions
ploting_cnt_amt(train, 'addr2')

In [ ]:
print(' addr1 - has {} NA values'.format(train['addr1'].isna().sum()))
print(' addr2 - has {} NA values'.format(train['addr2'].isna().sum()))

#### P Emaildomain

In [ ]:
train.loc[train['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'

train.loc[train['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo Mail'
train.loc[train['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
train.loc[train.P_emaildomain.isin(train.P_emaildomain\
                                         .value_counts()[train.P_emaildomain.value_counts() <= 500 ]\
                                         .index), 'P_emaildomain'] = "Others"
train.P_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
# domínio dos emails
ploting_cnt_amt(train, 'P_emaildomain')

#### R-Email Domain

In [ ]:
train.loc[train['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

train.loc[train['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
train.loc[train['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
train.loc[train.R_emaildomain.isin(train.R_emaildomain\
                                         .value_counts()[train.R_emaildomain.value_counts() <= 300 ]\
                                         .index), 'R_emaildomain'] = "Others"
train.R_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
ploting_cnt_amt(train, 'R_emaildomain')

#### C1-C14

In [ ]:
resumetable(train[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 
                    'C11', 'C12', 'C13', 'C14']])

In [ ]:
train[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
                      'C9', 'C10', 'C11', 'C12', 'C13', 'C14']].describe()

In [ ]:
train.loc[train.C1.isin(train.C1\
                              .value_counts()[train.C1.value_counts() <= 400 ]\
                              .index), 'C1'] = "Others"

In [ ]:
# C1 Distribution Plot
ploting_cnt_amt(train, 'C1')

In [ ]:
train.loc[train.C2.isin(train.C2.value_counts()[train.C2.value_counts() <= 350]
                             .index), 'C2'] = 'Others'

In [ ]:
# C2 Distribution Plot
ploting_cnt_amt(train, 'C2')

In [ ]:
# C3 Distribution Plot
ploting_cnt_amt(train, 'C3')

In [ ]:
train.loc[train.C4.isin(train.C4.value_counts()[train.C4.value_counts() <= 350]
                             .index), 'C4'] = 'Others'

In [ ]:
# C4 Distribution Plot
ploting_cnt_amt(train, 'C4')

In [ ]:
train.loc[train.C5.isin(train.C5.value_counts()[train.C5.value_counts() <= 350]
                             .index), 'C5'] = 'Others'

In [ ]:
# C5 Distribution Plot
ploting_cnt_amt(train, 'C5')

In [ ]:
train.loc[train.C6.isin(train.C6.value_counts()[train.C6.value_counts() <= 500]
                             .index), 'C6'] = 'Others'

In [ ]:
# C6 Distribution Plot
ploting_cnt_amt(train, 'C6')

In [ ]:
train.loc[train.C7.isin(train.C7.value_counts()[train.C7.value_counts() <= 500]
                             .index), 'C7'] = 'Others'

In [ ]:
# C7 Distribution Plot
ploting_cnt_amt(train, 'C7')

In [ ]:
train.loc[train.C8.isin(train.C8.value_counts()[train.C8.value_counts() <= 500]
                             .index), 'C8'] = 'Others'

In [ ]:
# C8 Distribution Plot
ploting_cnt_amt(train, 'C8')

In [ ]:
train.loc[train.C9.isin(train.C9.value_counts()[train.C9.value_counts() <= 500]
                             .index), 'C9'] = 'Others'
# C9 Distribution Plot
ploting_cnt_amt(train, 'C9')

In [ ]:
train.loc[train.C10.isin(train.C10.value_counts()[train.C10.value_counts() <= 500]
                             .index), 'C10'] = 'Others'
# C10 Distribution Plot
ploting_cnt_amt(train, 'C10')

In [ ]:
train.loc[train.C11.isin(train.C11.value_counts()[train.C11.value_counts() <= 500]
                             .index), 'C11'] = 'Others'
# C11 Distribution Plot
ploting_cnt_amt(train, 'C11')

In [ ]:
train.loc[train.C12.isin(train.C12.value_counts()[train.C12.value_counts() <= 500]
                             .index), 'C12'] = 'Others'
# C12 Distribution Plot
ploting_cnt_amt(train, 'C12')

In [ ]:
train.loc[train.C13.isin(train.C13.value_counts()[train.C13.value_counts() <= 10000]
                             .index), 'C13'] = 'Others'
# C13 Distribution Plot
ploting_cnt_amt(train, 'C13')

In [ ]:
train.loc[train.C14.isin(train.C14.value_counts()[train.C14.value_counts() <= 500]
                             .index), 'C14'] = 'Others'
# C14 Distribution Plot
ploting_cnt_amt(train, 'C14')

#### Dist1 and Dist2

In [ ]:
resumetable(train[['dist1', 'dist2']])

In [ ]:
train[['dist1', 'dist2']].describe()

In [ ]:
train.loc[train.dist1.isin(train.dist1.value_counts()[train.dist1.value_counts() <= 500]
                             .index), 'dist1'] = 'Others'
# dist1 Distribution Plot
ploting_cnt_amt(train, 'dist1')

In [ ]:
train.loc[train.dist2.isin(train.dist2.value_counts()[train.dist2.value_counts() <= 500]
                             .index), 'dist2'] = 'Others'
# dist2 Distribution Plot
ploting_cnt_amt(train, 'dist2')

#### D1-D15

In [ ]:
resumetable(train[['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']])

In [ ]:
train[['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']].describe()

In [ ]:
train.loc[train.D1.isin(train.D1.value_counts()[train.D1.value_counts() <= 500]
                             .index), 'D1'] = 'Others'
# D1 Distribution Plot
ploting_cnt_amt(train, 'D1')

In [ ]:
train.loc[train.D2.isin(train.D2.value_counts()[train.D2.value_counts() <= 500]
                             .index), 'D2'] = 'Others'
# D2 Distribution Plot
ploting_cnt_amt(train, 'D2')

In [ ]:
train.loc[train.D3.isin(train.D3.value_counts()[train.D3.value_counts() <= 500]
                             .index), 'D3'] = 'Others'
# D3 Distribution Plot
ploting_cnt_amt(train, 'D3')

In [ ]:
train.loc[train.D4.isin(train.D4.value_counts()[train.D4.value_counts() <= 1000]
                             .index), 'D4'] = 'Others'
# D4 Distribution Plot
ploting_cnt_amt(train, 'D4')

In [ ]:
train.loc[train.D5.isin(train.D5.value_counts()[train.D5.value_counts() <= 1000]
                             .index), 'D5'] = 'Others'
# D5 Distribution Plot
ploting_cnt_amt(train, 'D5')

In [ ]:
train.loc[train.D6.isin(train.D6.value_counts()[train.D6.value_counts() <= 500]
                             .index), 'D6'] = 'Others'
# D6 Distribution Plot
ploting_cnt_amt(train, 'D6')

In [ ]:
train.loc[train.D7.isin(train.D7.value_counts()[train.D7.value_counts() <= 500]
                             .index), 'D7'] = 'Others'
# D7 Distribution Plot
ploting_cnt_amt(train, 'D7')

In [ ]:
train.loc[train.D8.isin(train.D8.value_counts()[train.D8.value_counts() <= 500]
                             .index), 'D8'] = 'Others'
# D8 Distribution Plot
ploting_cnt_amt(train, 'D8')

In [ ]:
train.loc[train.D9.isin(train.D9.value_counts()[train.D9.value_counts() <= 500]
                             .index), 'D9'] = 'Others'
# D9 Distribution Plot
ploting_cnt_amt(train, 'D9')

In [ ]:
train.loc[train.D10.isin(train.D10.value_counts()[train.D10.value_counts() <= 1000]
                             .index), 'D10'] = 'Others'
# D10 Distribution Plot
ploting_cnt_amt(train, 'D10')

In [ ]:
train.loc[train.D11.isin(train.D11.value_counts()[train.D11.value_counts() <= 1000]
                             .index), 'D11'] = 'Others'
# D11 Distribution Plot
ploting_cnt_amt(train, 'D11')

In [ ]:
train.loc[train.D12.isin(train.D12.value_counts()[train.D12.value_counts() <= 500]
                             .index), 'D12'] = 'Others'
# D12 Distribution Plot
ploting_cnt_amt(train, 'D12')

In [ ]:
train.loc[train.D13.isin(train.D13.value_counts()[train.D13.value_counts() <= 500]
                             .index), 'D13'] = 'Others'
# D13 Distribution Plot
ploting_cnt_amt(train, 'D13')

In [ ]:
train.loc[train.D14.isin(train.D14.value_counts()[train.D14.value_counts() <= 500]
                             .index), 'D14'] = 'Others'
# D14 Distribution Plot
ploting_cnt_amt(train, 'D14')

In [ ]:
train.loc[train.D15.isin(train.D15.value_counts()[train.D15.value_counts() <= 1000]
                             .index), 'D15'] = 'Others'
# D15 Distribution Plot
ploting_cnt_amt(train, 'D15')

#### V1 - V339

In [ ]:
v_cols = [c for c in train if c[0] == 'V']
train[v_cols].head()

In [ ]:
train[v_cols].describe()

In [ ]:
train['v_mean'] = train[v_cols].mean(axis = 1)

In [ ]:
# fazendo o mesmo para a base de teste
test['v_mean'] = test[v_cols].mean(axis = 1)

#### Time Delta Feature

In [ ]:
train['TransactionDT'].plot(kind = 'hist', 
                            figsize = (15, 5), 
                            label = 'train', 
                            bins = 50, 
                            title = 'Train vs Test TransactionDT Distribution')

test['TransactionDT'].plot(kind = 'hist', 
                           label = 'test', 
                           bins = 50)
plt.legend()
plt.show()

Checando se as fraudes apresentam horários específicos de maior taxa

In [ ]:
# Converting to Total Days, Weekdays and Hours
# We will use the first date as 2017-12-01 and use the delta time to compute datetime features

import datetime

START_DATE = '2017-12-01'
start_date = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
train["Date"] = train['TransactionDT'].apply(lambda x: (start_date + datetime.timedelta
                                                             (seconds = x)))

train['_Weekdays'] = train['Date'].dt.dayofweek
train['_Hours'] = train['Date'].dt.hour
train['_Days'] = train['Date'].dt.day

In [ ]:
# Top Days With Highest Total Transaction Amount¶
ploting_cnt_amt(train, '_Days')

In [ ]:
# Ploting WeekDays Distributions
ploting_cnt_amt(train, '_Weekdays')

In [ ]:
# Ploting Hours Distributions
ploting_cnt_amt(train, '_Hours')

In [ ]:
# Top Days With Highest Total Transaction Amount¶
#pd.options.display.max_rows = 999
#pd.set_option('display.max_rows', None)
#train['Date']

In [ ]:
# fazendo o mesmo para base teste

START_DATE = '2017-12-01'
start_date = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
test["Date"] = test['TransactionDT'].apply(lambda x: (start_date + datetime.timedelta
                                                             (seconds = x)))

test['_Weekdays'] = test['Date'].dt.dayofweek
test['_Hours'] = test['Date'].dt.hour
test['_Days'] = test['Date'].dt.day

In [ ]:
#pd.set_option('display.max_rows', None)
#test['Date']

In [ ]:
pd.reset_option('display.max_rows')

# Cálculo tempo total do script

In [ ]:
t1 = time.time()
print("Script took {:.2} s".format(t1 - t0))